In [ ]:
# https://www.geeksforgeeks.org/scrape-google-search-results-using-python-beautifulsoup/
# https://practicaldatascience.co.uk/data-science/how-to-scrape-google-search-results-using-python

In [ ]:
# !pip install requests_html

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd /home/ec2-user/SageMaker/just-a-recipe-generator/

/home/ec2-user/SageMaker/just-a-recipe-generator


In [6]:
import requests
import json
import bs4
import pickle
from bs4 import BeautifulSoup
import urllib
import requests
import urllib
import pandas as pd
from requests_html import HTML
from requests_html import HTMLSession

from src.data.web_scraping import get_source
from src.data.scrape_glutenfreeonashoestring import scrape_shoestring_page, scrape_all_shoestring_pages
from src.data.pickling import save_pickle, load_pickle

In [ ]:
all_recipes = scrape_all_shoestring_pages()

In [7]:
all_recipes = load_pickle("data/raw/shoestring_recipes.pickle")

Loading data/raw/shoestring_recipes.pickle for consumption...


In [13]:
len(all_recipes)

1078

In [12]:
all_recipes["https://glutenfreeonashoestring.com/d-y-friday-homemade-hot-chocolate-mix/"]

{'Homemade Gluten Free Hot Chocolate Mix [White Chocolate &amp; Dairy Free Options]': {'@context': 'http://schema.org/',
  '@type': 'Recipe',
  'name': 'Homemade Gluten Free Hot Chocolate Mix [White Chocolate &amp; Dairy Free Options]',
  'author': {'@type': 'Person', 'name': 'Nicole Hunn'},
  'description': 'Why settle for boring and bland cocoa when you can enjoy creamy and delicious gluten free hot chocolate? Try these easy recipes to see the difference.',
  'datePublished': '2022-01-05T10:16:38+00:00',
  'image': ['https://glutenfreeonashoestring.com/wp-content/uploads/2015/01/Hot-Chocolate-in-hand-315x315-1.jpg',
   'https://glutenfreeonashoestring.com/wp-content/uploads/2015/01/Hot-Chocolate-in-hand-315x315-1-315x270.jpg'],
  'recipeYield': ['2', '2 cups mix'],
  'prepTime': 'PT10M',
  'cookTime': 'PT10M',
  'recipeIngredient': ['3/4 cup unsweetened cocoa powder ((preferably Dutch-processed))',
   '1/2 cup powdered milk',
   '2 teaspoons gluten free sweet rice flour',
   '1/2 tea

## Scrape all recipes

Need to use selenium or come up with different urls

### Not using below

In [ ]:
response = get_source(url)

In [ ]:
html = requests.get(url)
soup = BeautifulSoup(html.text, 'lxml')

In [ ]:
soup.find("div", {"class": "facetwp-facet facetwp-facet-pagination facetwp-type-pager"})

In [ ]:
soup.find("div", {"class": "facetwp-pager"})

In [ ]:
soup_text = str(soup)

In [ ]:
type(soup_text)

In [ ]:
"facetwp-facet" in soup_text

In [ ]:
len(soup_text) / 200

In [ ]:
soup_list = []
beginning = 0
end = 600
interval = 200

while beginning < len(soup_text):
    start = beginning + interval
    soup_list.append(soup_text[beginning:start])
    beginning = beginning + interval

In [ ]:
len(soup_list)

In [ ]:
for item in soup_list:
    if "pagination facetwp-type-pager" in item:
        item_index = soup_list.index(item)
        print(item_index, item)

In [ ]:
soup_list[503:505]

In [ ]:
709*200

In [ ]:
pagination_section = soup_text[100600:101000]
print(pagination_section)

In [ ]:
pages_section = soup_text[141300:142300]
print(pages_section)

In [ ]:
for item in soup_list:
    if "54" in item:
        item_index = soup_list.index(item)
        print(item_index, item)

In [ ]:
# https://betterprogramming.pub/how-to-scrape-multiple-pages-of-a-website-using-a-python-web-scraper-4e2c641cff8